In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating project at `~/Downloads/notebooks/makie_demos`


In [4]:
using GLMakie
using Colors
using GeometryBasics
using Rotations
using FileIO

GLMakie.activate!(inline=false)

In [10]:
# convert standrard quaternion, euler angle to makie quaternin type
function to_makie_quaternion(R_quat::QuatRotation)::Quaternion
    return  Quaternion(R_quat.q.s, R_quat.q.v1,R_quat.q.v2,R_quat.q.v3)
end

function to_makie_quaternion(R_euler::RotZYX)::Quaternion
    
    # Convert the rotation to unit quaternion
    R_quat = QuatRotation(R_euler)
    
    return  Quaternion(R_quat.q.s, R_quat.q.v1,R_quat.q.v2,R_quat.q.v3)
end

to_makie_quaternion (generic function with 2 methods)

### Loading a mesh

In [70]:
# load the model file
airplane_stl = load(assetpath(pwd() * "/assets/airplane.stl"));
tailsitter_stl = load(assetpath(pwd() * "/assets/tailsitter.stl"));
crazyflie_stl = load(assetpath(pwd() * "/assets/cf2_assembly.obj"));

In [54]:
struct Position
    x::Float64
    y::Float64
    z::Float64
end

# Orientation = QuatRotation

    
mutable struct RigidBody3
    position::Position
    orientation::QuatRotation
    
    # default constructor
    function RigidBody3()
        position = Position(0,0,0);
        orientation = QuatRotation(1,0,0,0);
        
        new(position,orientation)
    end
    
    # position and orientation
    function RigidBody3(position_::Position, orientation_::QuatRotation)
        
        new(position_,orientation_)
    end
    
    # position only
    function RigidBody3(position_::Position)

        orientation = QuatRotation(1,0,0,0);
        
        new(position_,orientation)
    end
    
    # orientation only
    function RigidBody3(orientation_::QuatRotation)
        position = Position(0,0,0);

        new(position,orientation_)
    end
end

RigidBody = RigidBody3

RigidBody3

In [55]:
b1 = RigidBody3()
b2 = RigidBody3(Position(1,1,0))
b3 = RigidBody3(QuatRotation(0,0,0,1))
b4 = RigidBody3(Position(1,1,0), QuatRotation(0,0,0,1))

RigidBody3(Position(1.0, 1.0, 0.0), [-1.0 0.0 0.0; 0.0 -1.0 0.0; 0.0 0.0 1.0])

In [69]:
set_theme!(
#     font = "Arial", # inherited by layoutables if not overridden
    fontsize = 25, # inherited by layoutables if not overridden
)

f = Figure(resolution = (1080, 720))

# Top level
g_left = f[1, 1] = GridLayout(alignmode=Outside(50))
g_right = f[1, 2] = GridLayout(alignmode=Outside(30))

Box(f[1, 2], color = (:red, 0.2), strokewidth = 0)

# force 3d visualizer to have an aspect ratio of 1
colsize!(f.layout, 1, Aspect(1, 1.0))

# # Column size adjust
colsize!(g_left, 1,  Auto(1))
colsize!(g_right, 1,  Auto(1))

# # Left grid
# ax_left = Axis3(g_left[1, 1])

# axis for airplane visualization
ax1 = Axis3(g_left[1, 1],
    title = "Brain activation",
    limits =  (-2,2,-2,2,-2,2),
    aspect = (1,1,1),
    xlabel="x axis", xlabelsize=40,
    ylabel="y axis", ylabelsize=40,
    zlabel="z axis", zlabelsize=40,
    halign =:left
)

m = mesh!(ax1, crazyflie_stl , color=:red)

scale!(m, 10.5, 10.5, 10.5)

# center model at the origin
translate!(m, Vec3f(0.0, 0.0, 0.0))

# orient along positive x axis
rotate!(m, Vec3f(0, 1, 0), 0) # 0.5 rad around the y axis


# Right Grids - plots
g_right_plots = GridLayout()
g_right_widgets = GridLayout()

plot_1 = Axis(f)
plot_2 = Axis(f)

g_right_plots[2,1] = plot_1
g_right_plots[3,1] = plot_2
g_right[1, 1] = g_right_plots

# Right Grids - widgets

# slider grid for setting attitude
attitude_slider = SliderGrid(
    f,
    (label = "Roll", range = -pi:0.01:pi, format = "{:.2f} rad", startvalue = 0.0, linewidth = 25.0),
    (label = "Pitch", range = -pi:0.01:pi, format = "{:.2f} rad", startvalue = 0.0, linewidth = 25.0),
    (label = "Yaw", range = -pi:0.01:pi, format = "{:.2f} rad", startvalue = 0.0, linewidth = 25.0),
    width = Auto(),
    tellheight = false,
    halign =:left
)

# attitude reset button
attitude_reset_btn = Button(f, label = "Reset Attitude", tellwidth=false)

g_right_widgets[1,1] = attitude_slider
g_right_widgets[2,1] = attitude_reset_btn
g_right[2, 1] = g_right_widgets

# Observables
attitude_slider_obs = [s.value for s in attitude_slider.sliders]


# events

# Attitude button clicked
on(attitude_reset_btn.clicks) do clicks
    # reset attitude
    rotate!(m, Quaternion(1,0,0,0))  
    
    # reset the attitude control sliders
    set_close_to!(attitude_slider.sliders[1], attitude_slider.sliders[1].startvalue[])
    set_close_to!(attitude_slider.sliders[2], attitude_slider.sliders[1].startvalue[])
    set_close_to!(attitude_slider.sliders[3], attitude_slider.sliders[1].startvalue[])
end


bars = lift(attitude_slider_obs...) do angles...
    
    # create ZYX euler angle representation 
    R_euler = RotZYX(angles[1],angles[2],angles[3])
    
    # Convert the rotation to unit quaternion
    R_quat = QuatRotation(R_euler)
    
    R_quat_makie = Quaternion(R_quat.q.s, R_quat.q.v1,R_quat.q.v2,R_quat.q.v3)
    
    # apply rotation
    rotate!(m,  R_quat_makie)     
end

trim!(f.layout)

display(f)

GLMakie.Screen(...)

# Airplane

In [57]:
set_theme!(
#     font = "Arial", # inherited by layoutables if not overridden
    fontsize = 25, # inherited by layoutables if not overridden
)

f = Figure(resolution = (1080, 720))

# Top level
g_left = f[1, 1] = GridLayout(alignmode=Outside(50))
g_right = f[1, 2] = GridLayout(alignmode=Outside(30))

Box(f[1, 2], color = (:red, 0.2), strokewidth = 0)

# force 3d visualizer to have an aspect ratio of 1
colsize!(f.layout, 1, Aspect(1, 1.0))

# # Column size adjust
colsize!(g_left, 1,  Auto(1))
colsize!(g_right, 1,  Auto(1))

# # Left grid
# ax_left = Axis3(g_left[1, 1])

# axis for airplane visualization
ax1 = Axis3(g_left[1, 1],
    title = "Brain activation",
    limits =  (-2,2,-2,2,-2,2),
    aspect = (1,1,1),
    xlabel="x axis", xlabelsize=40,
    ylabel="y axis", ylabelsize=40,
    zlabel="z axis", zlabelsize=40,
    halign =:left
)

tailsitter_mesh = mesh!(ax1, tailsitter_stl , color=:red)

scale!(tailsitter_mesh, 0.001, 0.001, 0.001)

# orient along positive x axis
rotate!(tailsitter_mesh, Vec3f(0, 0, 1), pi) # 0.5 rad around the y axis


# Right Grids - plots
g_right_plots = GridLayout()
g_right_widgets = GridLayout()
g_right_display = GridLayout()

plot_1 = Axis(f)
plot_2 = Axis(f)

g_right_plots[2,1] = plot_1
g_right_plots[3,1] = plot_2

g_right[1, 1] = g_right_plots
g_right[2, 1] = g_right_display
g_right[3, 1] = g_right_widgets


# Right Grids - widgets

# slider grid for setting attitude
attitude_slider = SliderGrid(
    f,
    (label = "Roll", range = -pi:0.01:pi, format = "{:.2f} rad", startvalue = 0.0, linewidth = 25.0),
    (label = "Pitch", range = -pi:0.01:pi, format = "{:.2f} rad", startvalue = 0.0, linewidth = 25.0),
    (label = "Yaw", range = -pi:0.01:pi, format = "{:.2f} rad", startvalue = 0.0, linewidth = 25.0),
    width = Auto(),
    tellheight = false,
    halign =:left
)

# attitude reset button
attitude_reset_btn = Button(f, label = "Reset Attitude", tellwidth=false)

# Attitude display label

attitude_display = Label(f,
    "Quaternion\n\nq_w:\t0.0\nq_x:\t0.0\nq_y:\t0.0\nq_z:\t0.0 \n",
    justification = :left,
    lineheight = 0.9,
    fontsize = 30
)

g_right_display[1,1:2] = attitude_display

g_right_widgets[1,1] = attitude_slider
g_right_widgets[2,1] = attitude_reset_btn



# Observables
attitude_slider_obs = [s.value for s in attitude_slider.sliders]


# events

# Attitude button clicked
on(attitude_reset_btn.clicks) do clicks
    # reset attitude
    rotate!(tailsitter_mesh, Quaternion(1,0,0,0))  
    
    # reset the attitude control sliders
    set_close_to!(attitude_slider.sliders[1], attitude_slider.sliders[1].startvalue[])
    set_close_to!(attitude_slider.sliders[2], attitude_slider.sliders[1].startvalue[])
    set_close_to!(attitude_slider.sliders[3], attitude_slider.sliders[1].startvalue[])
end


bars = lift(attitude_slider_obs...) do angles...
    
    # create ZYX euler angle representation 
    R_euler = RotZYX(angles[1],angles[2],angles[3])
    
    # Convert the rotation to unit quaternion
    R_quat = QuatRotation(R_euler)
    
    R_quat_makie = to_makie_quaternion(R_euler)
    
    # apply rotation
    rotate!(tailsitter_mesh,  R_quat_makie)    
    
    # update attitude display
    attitude_display.text = "Quaternion\n\nq_w:\t" * string(round(R_quat.q.s,digits=2)) * "\nq_x:\t" * string(round(R_quat.q.v1,digits=2)) * "\nq_y:\t" * string(round(R_quat.q.v2,digits=2)) * "\nq_z:\t" * string(round(R_quat.q.v3,digits=2)) * "\n"
end
 
trim!(f.layout)

display(f)

GLMakie.Screen(...)

# Utility function